In [1]:
import cvxpy as cp
import numpy as np
import time
import clarabel
import torch
from zap.admm import ADMMSolver
from zap.conic.cone_bridge import ConeBridge
import scipy.sparse as sp
from zap.conic.cone_utils import generate_max_flow_problem, is_valid_network, get_standard_conic_problem


/home/groups/ramr/akshay81/poetry_cache/virtualenvs/zap-eFcx8M23-py3.12/lib/python3.12/site-packages/pypsa/linopf.py:1253: SyntaxWarning: invalid escape sequence '\ '
  n.objective_f.write("\ LOPF \n\nmin\nobj:\n")


In [3]:
## Create a large problem that is valid 
n = 10000
seed = 42
valid_source_sink_path = False
quad_obj = False

while (not valid_source_sink_path):
    problem, adj, inc = generate_max_flow_problem(n, quad_obj=quad_obj, seed=seed)
    valid_source_sink_path = is_valid_network(adj)
    if (not valid_source_sink_path):
        seed += 1
nnz = float(adj.nnz)
total_possible_edges = float(adj.shape[0]*(adj.shape[0] - 1))
density = nnz/total_possible_edges
sparsity = 1 - density
num_variables = sum(np.prod(var.shape) for var in problem.variables())
num_constraints = sum(constraint.size for constraint in problem.constraints)
print(f'Generated a valid network with {n} nodes using seed {seed}')
print(f"Actual Number of Edges: {nnz}")
print(f"Total Possible Edges: {total_possible_edges}")
print(f"Graph sparsity: {sparsity}")
print(f"Graph density: {density}")
print(f"Number of Variables: {num_variables}")
print(f"Number of Constraints: {num_constraints}")

Generated a valid network with 10000 nodes using seed 42
Actual Number of Edges: 183993.0
Total Possible Edges: 99990000.0
Graph sparsity: 0.9981598859885988
Graph density: 0.0018401140114011401
Number of Variables: 183993
Number of Constraints: 193993


In [18]:
# Solve the problem in a standard way using CVXPY
start_time = time.time()
# result = problem.solve(solver=cp.SCIPY, scipy_options={'method': "highs-ds"}, verbose=True)
# result = problem.solve(solver=cp.CLARABEL, verbose=True, tol_gap_abs=1e-3, tol_gap_rel=1e-3)
result = problem.solve(solver=cp.CLARABEL, verbose=True, max_iter=10)
end_time = time.time()
solve_time = end_time - start_time
obj_val = problem.value
problem_status = problem.status
num_iters = problem.solver_stats.num_iters
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Apr 08 05:53:49 PM: Your problem has 183993 variables, 193993 constraints, and 0 parameters.
(CVXPY) Apr 08 05:53:49 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 08 05:53:49 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 08 05:53:49 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 08 05:53:49 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 08 05:53:49 PM: Compiling problem (target solver=CL

In [20]:
problem.solver_stats

SolverStats(solver_name='CLARABEL', solve_time=972.919782039, setup_time=None, num_iters=10, extra_stats=None)

In [14]:
# Get conic problem form so we can (i) solve standard conic form and (ii) solve using ZAP
cone_params, data, cones = get_standard_conic_problem(problem, solver=cp.CLARABEL)

In [10]:
cone_params

{'A': <Compressed Sparse Column sparse matrix of dtype 'float64'
 	with 551979 stored elements and shape (193993, 183993)>,
 'b': array([0.e+00, 0.e+00, 0.e+00, ..., 5.e+00, 5.e+00, 1.e+04]),
 'c': array([ 0.,  0.,  0., ...,  0.,  0., -1.]),
 'K': {'z': 10000, 'l': 183993, 'q': [], 'ep': 0, 's': []}}

In [20]:
start_time = time.time()
A_cl = cone_params['A']
b_cl = cone_params['b']
q_cl = cone_params['c']
P_cl = sp.csc_matrix(np.zeros((A_cl.shape[1], A_cl.shape[1])))
cones_cl = [clarabel.ZeroConeT(cones['z']), clarabel.NonnegativeConeT(cones['l'])]
settings = clarabel.DefaultSettings()
settings.verbose=False
settings.max_iter = 5000
end_time = time.time()
solve_time = end_time - start_time
print(f"Time taken: {solve_time:.4f} seconds")


Time taken: 221.2601 seconds


In [14]:
# Solve the conic form using CLARABEL
A_cl = cone_params['A']
b_cl = cone_params['b']
q_cl = cone_params['c']
P_cl = sp.csc_matrix(np.zeros((A_cl.shape[1], A_cl.shape[1])))
cones_cl = [clarabel.ZeroConeT(cones['z']), clarabel.NonnegativeConeT(cones['l'])]
settings = clarabel.DefaultSettings()
settings.verbose=False
settings.max_iter = 5000
start_time = time.time()
solver = clarabel.DefaultSolver(P_cl,q_cl,A_cl,b_cl,cones_cl, settings)
end_time = time.time()
soln = solver.solve()
solve_time = end_time - start_time
obj_val = soln.obj_val
problem_status = soln.status
num_iters = soln.iterations
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")

Objective value: -157.99999999370752
Time taken: 1.2281 seconds
Number of iterations: 12
Status: Solved


In [15]:
# Solve the conic form using ZAP
machine = 'cuda'
dtype = torch.float32
cone_bridge = ConeBridge(cone_params)
cone_admm_devices = [d.torchify(machine=machine, dtype=dtype) for d in cone_bridge.devices]
cone_admm = ADMMSolver(
    machine=machine,
    dtype=dtype,
    atol=1e-9,
    rtol=1e-9,
    num_iterations=10000,
)
start_time = time.time()
cone_solution_admm, cone_history_admm = cone_admm.solve(
    net=cone_bridge.net, devices=cone_admm_devices, time_horizon=cone_bridge.time_horizon
)
end_time = time.time()
solve_time = end_time - start_time
obj_val = cone_solution_admm.objective
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")



Did not converge. Ran for 10000 iterations.
Objective value: -158.00062561035156
Time taken: 43.0372 seconds


In [21]:
## Solve using CVXPY and CLARABEL from Zap formulation
start_time = time.time()
outcome = cone_bridge.solve(solver=cp.CLARABEL)
end_time = time.time()
solve_time = end_time - start_time
obj_val = outcome.problem.value
problem_status = outcome.problem.status
num_iters = outcome.problem.solver_stats.num_iters
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")



Objective value: -157.99999999370155
Time taken: 987.8816 seconds
Number of iterations: 12
Status: optimal
